In [ ]:
import sys
import os
from pathlib import Path

import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from src.utils.data_extracting import get_all_account_statement_files, extract_text_from_pdf, get_balance_of_account, get_all_transactions
from src.utils.data_loading import update_google_sheet
from src.utils.data_transforming import extract_transaction_info, add_new_row
from src.utils.google_connection import set_up_google_connection

# Get files

In [ ]:
credentials_path = Path('../credentials/cool-plasma-452619-v4-feb20b70d461.json')
downloads_path = Path.home() / 'Downloads'

In [ ]:
files = get_all_account_statement_files(downloads_path)
files

In [ ]:
for file in files:
    print(file)

In [ ]:
file_path = files[1]
full_pdf_text = extract_text_from_pdf(file_path)
lines = full_pdf_text.split('\n')

In [ ]:
acc_balance_old = get_balance_of_account(lines, 'alter Kontostand')
acc_balance_new = get_balance_of_account(lines, 'neuer Kontostand')
print(f'Old acc value, Value: {acc_balance_old[0]}€ - Line-Index: {acc_balance_old[1]}')
print(f'New acc value, Value: {acc_balance_new[0]}€ - Line-Index: {acc_balance_new[1]}')

all_transactions = get_all_transactions(lines, acc_balance_old[1], acc_balance_new[1])
print(f'Count of transactions: {len(all_transactions)}')

# Google Connection

In [ ]:
client, service = set_up_google_connection(credentials_path)

In [ ]:
spreadsheet = client.open_by_key("1OnrW1foE-1lOtgfxBv2Y5qqJSDnW4hiYeLpScjgFKxM") # TODO: Put in secrets/env

sheet_incomes = spreadsheet.worksheet('Einnahmen')
sheet_expenses = spreadsheet.worksheet('Ausgaben')

df_expenses = pd.DataFrame(sheet_expenses.get_all_values())
df_incomes = pd.DataFrame(sheet_incomes.get_all_values())

In [ ]:
df_expenses.columns = df_expenses.iloc[0]
df_expenses = df_expenses[1:].reset_index(drop=True)

df_incomes.columns = df_incomes.iloc[0]
df_incomes = df_incomes[1:].reset_index(drop=True)

gsheets = {'Expense': df_expenses, 'Income': df_incomes}

In [ ]:
gsheets['Income']

In [ ]:
gsheets['Expense']

In [ ]:
for transaction_index in range(len(all_transactions)):
    transaction = all_transactions[transaction_index]

    transaction_type, df, name, transaction_value, month = (
        extract_transaction_info(transaction, gsheets)
    )

    add_new_row(
        df,
        name,
        month,
        transaction_value,
        transaction_type,
        gsheets,
        general_account=True,
    )

In [ ]:
gsheets['Income']

In [ ]:
gsheets['Expense']

# Load data into Google Sheet

In [ ]:
update_google_sheet(sheet_expenses, gsheets['Expense'])
update_google_sheet(sheet_incomes, gsheets['Income'])
print('✅ All changes saved to Google Sheets!')